In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
import numpy as np
import statistics as stats
import sklearn
import re
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

pd.set_option('display.max_columns', None)



%matplotlib inline

In [2]:
df = pd.read_csv('fifa21_train.csv')


In [3]:
# Define a function


def preprocess(data):

    # Fix the weights (as kg)
    Weight = [] 
    for i in data['Weight']:
        Weight.append(str(i).replace('lbs' , ''))

    data['Weight'] = Weight
    data['Weight'] = data['Weight'].astype(np.number)
    data['Weight'] = data['Weight']/2.2046

    
    # Fix the heights (as cm)
    

    cmheight = []
    for item in data['Height']:
        item = str(item).replace('\"', '')
        feetinches = item.split('\'')
        cmheight.append(round(int(feetinches[0]) * 30.48) + int(feetinches[1]) * 2.54)
    data['Height'] = cmheight
        
    # Remove the stars from columns 'IR', 'W/F', and 'SM'
    newvalues = []
    for item in data['IR']:
        item = str(item).replace('★', '')
        item = item.strip()
        newvalues.append(int(item))
    data['IR']=newvalues
  
    newvalues = []
    for item in data['W/F']:
        item = str(item).replace('★', '')
        item = item.strip()
        newvalues.append(int(item))
    data['W/F']=newvalues
    
    newvalues = []
    for item in data['SM']:
        item = str(item).replace('★', '')
        item = item.strip()
        newvalues.append(int(item))
    data['SM']=newvalues
    
  
    # Transform the 'Contract' column to only give the end date of the contract 
#     contractend = []
#     for item in data['Contract']:
#         if 'Free' in item:
#             contractend.append('DELETE')
#         else:
#             item = item.replace(' On Loan', '')
#             item = item[-4:]
#             contractend.append(int(item))
        
#     data['Contract']=contractend

    # Transform the 'loan date end' column to an on loan (yes/no) column
    data['Loan Date End'] = data['Loan Date End'].fillna(0)

    loanyesno = []
    for item in data['Loan Date End']:
        if item == 0:
            loanyesno.append(item)
        else: 
            loanyesno.append(1)

    data['Loan Date End'] = loanyesno
    data = data.rename(columns={'Loan Date End':'On Loan'})

    data['On Loan']
    
    # Remove the plusses and minuses on the last few rows   
    toclean = ['LS', 'ST', 'RS', 'LW', 'LF', 'CF', 'RF', 'RW', 'LAM', 'CAM', 'RAM', 'LM', 'LCM', 'CM', 'RCM', 'RM', 'LWB', 'LDM', 'CDM', 'RDM', 'RWB', 'LB', 'LCB', 'CB', 'RCB', 'RB', 'GK']

    superlist = []

    for column in toclean: 
        values = []
        for item in data[column]:
            if item[1] == '-' or item[1] == '+':
                values.append(int(item[0]))
            else: 
                values.append(int(item[:2]))
        superlist.append(values)


    for i in range(len(toclean)):
        data[toclean[i]] = superlist[i]

    # Clean up currency cells by removing € sign and converting 'M' and 'K' to numbers
    
    newvalues1 = []
    for item in data['Value']:
        item = item.replace('€', '')
        if 'K' in item: 
            item = float(item.replace('K', ''))
            item = item * 1000
            newvalues1.append(int(item))
        elif 'M' in item: 
            item = float(item.replace('M', ''))
            item = item * 1000000      
            newvalues1.append(int(item))
        else: 
            item = int(item)
            newvalues1.append(int(item))
    data['Value']=newvalues1
    
    
    newvalues2 = []
    for item in data['Wage']:
        item = item.replace('€', '')
        if 'K' in item: 
            item = float(item.replace('K', ''))
            item *= 1000
            newvalues2.append(int(item))
        elif 'M' in item: 
            item = float(item.replace('M', ''))
            item *= 1000000      
            newvalues2.append(int(item))
        else: 
            item = int(item)
            newvalues2.append(int(item))
    data['Wage']=newvalues2
    
    
    newvalues3 = []
    for item in data['Release Clause']:
        item = item.replace('€', '')
        if 'K' in item: 
            item = float(item.replace('K', ''))
            item *= 1000
            newvalues3.append(int(item))
        elif 'M' in item: 
            item = float(item.replace('M', ''))
            item *= 1000000      
            newvalues3.append(int(item))
        else: 
            item = int(item)
            newvalues3.append(int(item))
    data['Release Clause']=newvalues3
    
    
    # Clean up Hits column (which contains some values with K in them)¶
    newhits = []
    for item in data['Hits']:
        if 'K' in item:
            item = item.replace('K','')
            item = float(item) * 1000
            newhits.append(int(item))
        else: 
            newhits.append(int(item))
    data['Hits'] = newhits

    # Dropping stuff
    
    # Drop columns with irrelevant attributes
    #Some atributes are used to calculate the rating per category, that we already have, so individual attributes are not necessary here

    # Different versions for commenting out and experimenting
    
#     # First, a version with only the summed up columns (Attacking etc)
#     data = data.drop(['Nationality','Club','Team & Contract', 'Position', 'ID','Growth','Joined', 'Contract','Name','Crossing','Finishing','Heading Accuracy','Short Passing','Volleys',
#            'Dribbling','Curve','FK Accuracy','Long Passing','Ball Control',
#            'Acceleration','Sprint Speed','Agility','Reactions','Balance',
#           'Shot Power','Jumping','Stamina','Strength','Long Shots',
#           'Aggression','Interceptions','Positioning','Vision','Penalties','Composure',
#           'Marking','Standing Tackle','Sliding Tackle',
#           'GK Diving','GK Handling','GK Kicking','GK Positioning','GK Reflexes'], axis=1)
    
    # Then, a version with only the component columns
    data = data.drop(['Nationality','Club','Team & Contract', 'Position', 'ID','Growth','Joined', 'Contract','Name','Crossing','Finishing','Heading Accuracy','Short Passing','Volleys',
           'Attacking', 'Skill', 'Movement', 'Power', 'Mentality', 'Defending', 'Defending'], axis=1)
    
    
    
    #data = data.drop(['Age', 'Height', 'Weight', 'On Loan', 'Value', 'Wage', 'Release Clause'], axis=1)
    #data = data.drop(['Value', 'Wage', 'Release Clause'], axis=1)
    #data = data.drop(['Age', 'Height', 'Weight'], axis=1)

    for column in data.columns: 
        data = data[data[column].isna()==False]

    #Drop the rows with less than 5% of NaN
#     data = data[data['A/W'].isna()==False]
#     data = data[data['D/W'].isna()==False]
    
    return data

data = preprocess(df)

In [4]:
# And a function to do the X-Y split
y = data['OVA']
X = data.drop(['OVA'], axis = 1)

#Split numerical and categorical data
X_num = X.select_dtypes(np.number)
X_cat = X.select_dtypes(object)


transformer = MinMaxScaler().fit(X_num)
# encoder = OneHotEncoder().fit(X_cat)


def xysplit(dataframe): 
    
    #Initial split (y will remain unchanged, X will undergo more operations)
    y = dataframe['OVA']
    X = dataframe.drop(['OVA'], axis = 1)
    
    #Split numerical and categorical data
    X_num = X.select_dtypes(np.number)
    X_cat = X.select_dtypes(object)

    #Normalize the numerical data
    x_normalized = transformer.transform(X_num)
    X_normal = pd.DataFrame(x_normalized, columns=X_num.columns)

    #Encode categorical data
    encoder = OneHotEncoder().fit(X_cat)

    encoded = encoder.transform(X_cat).toarray()

    # And get relevant headers for the encoded categorical data
    headers = []

    for category in encoder.categories_:
        for unit in category: 
            headers.append(unit)

    categ_encoded=pd.DataFrame(encoded, columns=headers)
    
    #Finally, concatenate the (normalized) numerical and (encoded) categorical data
    X = pd.concat([X_normal, categ_encoded], axis = 1)
    
    return X, y

In [5]:
data.head()

,Age,BP,Height,Weight,foot,On Loan,Value,Wage,Release Clause,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Acceleration,Sprint Speed,Agility,Reactions,Balance,Shot Power,Jumping,Stamina,Strength,Long Shots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,26,CM,174.86,73.029121,Right,0,525000,4000,801000,61,44.0,55,63,63,64,73,61.0,66,82.0,62,73.0,71,55,45,54,52.0,62.0,68.0,54,54.0,49,56,43.0,48,7,12,14,9,6,1682,357,4,2,High,Medium,1,69,51,63,63,51,60,3,58,58,58,61,62,62,62,61,63,63,63,63,63,63,63,63,59,59,59,59,59,58,54,54,54,58,15,64
1,30,ST,183.00,72.121927,Right,1,8500000,23000,0,83,78.0,72,63,79,83,83,88.0,75,75.0,74,81.0,75,74,68,54,33.0,78.0,72.0,76,70.0,35,20,22.0,55,11,7,14,7,16,1961,412,3,4,High,Low,2,83,75,68,82,33,71,44,77,77,77,77,77,77,77,77,76,76,76,76,68,68,68,76,57,53,53,53,57,53,48,48,48,53,18,77
2,33,CAM,162.16,60.782001,Right,0,9000000,49000,15300000,85,89.0,91,74,85,84,76,93.0,78,93.0,79,34.0,75,42,78,75,26.0,80.0,78.0,73,82.0,23,29,28.0,21,6,3,6,3,3,1925,404,4,4,High,Medium,2,80,77,78,86,27,56,73,73,73,73,80,79,79,79,80,80,80,80,79,74,74,74,79,59,56,56,56,59,53,41,41,41,53,12,80
3,22,CDM,177.40,68.946748,Right,0,275000,4000,694000,54,41.0,46,57,61,54,59,59.0,55,55.0,57,60.0,64,58,38,61,57.0,31.0,54.0,54,48.0,55,58,55.0,42,8,9,6,7,12,1527,329,2,2,Medium,Medium,1,57,44,54,57,57,60,7,50,50,50,51,51,51,51,51,53,53,53,53,56,56,56,53,56,58,58,58,56,57,58,58,58,57,14,59
4,23,CDM,179.94,68.039554,Right,0,725000,2000,1400000,64,44.0,45,61,66,66,66,61.0,62,69.0,61,34.0,81,61,43,66,60.0,55.0,64.0,49,58.0,58,61,66.0,52,8,9,15,5,15,1664,360,2,3,Low,Medium,1,66,44,60,64,60,66,4,56,56,56,59,59,59,59,59,61,61,61,62,63,63,63,62,64,64,64,64,64,63,61,61,61,63,15,65


In [6]:
X_num.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,11422.0,2.515558e+01,4.872713e+00,16.000000,21.000000,25.000000,2.900000e+01,4.300000e+01
Height,11422.0,1.810826e+02,7.124935e+00,154.540000,174.860000,179.940000,1.855400e+02,2.058600e+02
Weight,11422.0,7.508294e+01,7.169145e+00,53.070852,69.853942,74.843509,7.983308e+01,1.102241e+02
On Loan,11422.0,5.471896e-02,2.274408e-01,0.000000,0.000000,0.000000,0.000000e+00,1.000000e+00
Value,11422.0,2.584324e+06,5.317007e+06,0.000000,375000.000000,800000.000000,2.500000e+06,9.000000e+07
...,...,...,...,...,...,...,...,...
LCB,11422.0,5.309499e+01,1.456841e+01,16.000000,43.000000,55.000000,6.500000e+01,8.800000e+01
CB,11422.0,5.309499e+01,1.456841e+01,16.000000,43.000000,55.000000,6.500000e+01,8.800000e+01
RCB,11422.0,5.309499e+01,1.456841e+01,16.000000,43.000000,55.000000,6.500000e+01,8.800000e+01
RB,11422.0,5.426554e+01,1.354526e+01,14.000000,47.000000,57.000000,6.400000e+01,8.300000e+01


In [7]:
X_cat

,BP,foot,A/W,D/W
0,CM,Right,High,Medium
1,ST,Right,High,Low
2,CAM,Right,High,Medium
3,CDM,Right,Medium,Medium
4,CDM,Right,Low,Medium
...,...,...,...,...
11695,ST,Right,High,Low
11696,LB,Left,Medium,Medium
11697,GK,Right,Medium,Medium
11698,RM,Right,High,Medium


In [8]:
X, y = xysplit(data)

In [9]:
#Test train split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=25)

In [10]:
# Build the model 

lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)

LinearRegression()

In [11]:
# A function to generate scores to evaluate

def scores(y_data, predictions):
    print("r2 score:",r2_score(y_data, predictions))
    print("MSE score:",mean_squared_error(y_data, predictions))
    print("RMSE score:",np.sqrt(mean_squared_error(y_data, predictions)))
    print("MAE score:", mean_absolute_error(y_data, predictions))

In [12]:
# Get the R2 score for the training and test data

predictions = lm.predict(X_train)
predictions_test = lm.predict(X_test)

print("Training data")
scores(y_train, predictions)

print("Test data")
scores(y_test, predictions_test)

Training data
r2 score: 0.9181352798961773
MSE score: 3.8078768921065436
RMSE score: 1.9513782032467575
MAE score: 1.521439485402758
Test data
r2 score: 0.9200801566726462
MSE score: 3.848008148727584
RMSE score: 1.9616340506647982
MAE score: 1.5408412541028447


Validate

In [13]:
df2 = pd.read_csv('fifa21_validate.csv')

In [14]:
data2 = preprocess(df2)

In [17]:
# y2 here gives a meaningless value, but we'll ignore that and keep working with the original y

X2, y2 = xysplit(data2)

In [15]:
data2.head()

,Age,BP,Height,Weight,foot,On Loan,Value,Wage,Release Clause,Dribbling,Curve,FK Accuracy,Long Passing,Ball Control,Acceleration,Sprint Speed,Agility,Reactions,Balance,Shot Power,Jumping,Stamina,Strength,Long Shots,Aggression,Interceptions,Positioning,Vision,Penalties,Composure,Marking,Standing Tackle,Sliding Tackle,Goalkeeping,GK Diving,GK Handling,GK Kicking,GK Positioning,GK Reflexes,Total Stats,Base Stats,W/F,SM,A/W,D/W,IR,PAC,SHO,PAS,DRI,DEF,PHY,Hits,LS,ST,RS,LW,LF,CF,RF,RW,LAM,CAM,RAM,LM,LCM,CM,RCM,RM,LWB,LDM,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,23,CB,188.08,87.997823,Right,1,975000,5000,0,44,43.0,36,51,54,60,68,63.0,63,49.0,48,77.0,51,87,25,68,62.0,38.0,39.0,39,49.0,68,67,65.0,56,11,8,15,13,9,1551,334,2,2,Low,High,1,64,30,50,50,66,74,34,48,48,48,48,47,47,47,48,48,48,48,50,51,51,51,50,59,61,61,61,59,61,67,67,67,61,16,67
1,22,CAM,172.32,69.853942,Right,0,1200000,3000,2200000,76,68.0,56,60,70,77,72,83.0,64,79.0,76,62.0,63,47,75,72,26.0,64.0,64.0,60,65.0,19,23,19.0,53,14,13,9,9,8,1726,358,4,3,High,Low,1,74,67,62,74,24,57,12,64,64,64,68,68,68,68,68,68,68,68,67,61,61,61,67,49,47,47,47,49,45,38,38,38,45,17,68
2,19,GK,188.08,79.833076,Right,0,120000,500,249000,6,8.0,8,20,10,28,25,33.0,41,38.0,40,49.0,22,54,6,20,9.0,7.0,26.0,14,31.0,8,9,10.0,269,56,52,53,53,55,808,295,2,1,Medium,Medium,1,56,52,53,55,26,53,3,18,18,18,15,17,17,17,15,17,17,17,16,18,18,18,16,16,18,18,18,16,16,18,18,18,16,53,54
3,16,CDM,190.62,77.111494,Right,0,160000,500,464000,51,34.0,38,47,54,59,58,56.0,48,54.0,48,48.0,60,58,28,61,55.0,33.0,40.0,41,59.0,53,52,54.0,36,7,5,13,5,6,1381,303,3,2,Medium,Medium,1,58,34,47,52,53,59,6,46,46,46,47,46,46,46,47,47,47,47,49,49,49,49,49,53,54,54,54,53,53,54,54,54,53,11,55
4,24,CDM,188.08,77.111494,Right,0,2300000,13000,4300000,71,57.0,51,74,74,68,66,66.0,64,56.0,73,56.0,74,72,62,66,78.0,53.0,62.0,55,63.0,72,68,71.0,60,12,7,13,15,13,1864,407,4,2,Medium,Medium,1,67,62,68,70,69,71,45,63,63,63,66,66,66,66,66,68,68,68,67,70,70,70,67,70,72,72,72,70,69,68,68,68,69,18,70


In [18]:
predictions2 = lm.predict(X2)

In [19]:
scores(y2, predictions2)

r2 score: 0.9163663773026612
MSE score: 3.783595191325329
RMSE score: 1.9451465732240665
MAE score: 1.5240187275525912
